In [ ]:
ls /kaggle/input/india-shape-file/

In [ ]:
pip install pandas_bokeh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import pandas_bokeh
pandas_bokeh.output_notebook()
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show,output_file,save

In [ ]:
covid = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

In [ ]:
covid.columns

In [ ]:
covid.head()

In [ ]:
covid.describe()

In [ ]:
covid.isnull().sum()

In [ ]:
india = covid[covid['Country/Region'] == 'India']

In [ ]:
india.info()

In [ ]:
india['Province/State'].unique()

In [ ]:
india.isnull().sum()

In [ ]:
india['Province/State'] = india['Province/State'].fillna(value = "Unknown")

In [ ]:
india.isnull().sum()

In [ ]:
india['Province/State'].unique()

In [ ]:
india['Province/State'].value_counts()

In [ ]:
india['Province/State'].replace({'Other': 'Unknown'}, inplace=True)

In [ ]:
india['Province/State'].value_counts()

In [ ]:
india['Deaths'].sum()

In [ ]:
india.describe()

In [ ]:
india['Province/State'].replace({"Ladakh": "Jammu and Kashmir"}, inplace=True)

In [ ]:
india.corr()

In [ ]:
india['ObservationDate'] = pd.to_datetime(india['ObservationDate'])

In [ ]:
india = india.drop('Country/Region' ,axis = 1)

In [ ]:
confirmed_cases = pd.DataFrame(india.groupby(['Province/State'])['Confirmed'].nth(-1))
death_cases = pd.DataFrame(india.groupby(['Province/State'])['Deaths'].nth(-1))
recovered_cases = pd.DataFrame(india.groupby(['Province/State'])['Recovered'].nth(-1))

In [ ]:
death_cases.plot(kind = 'bar')

In [ ]:
india['Active'] = india["Confirmed"] - (india["Deaths"]+india["Recovered"])

In [ ]:
active_cases = pd.DataFrame(india.groupby(['Province/State'])['Active'].nth(-1))

In [ ]:
active_cases = active_cases.sort_values('Province/State')

In [ ]:
active_cases.index

In [ ]:
death_cases = death_cases.sort_values('Province/State')

In [ ]:
recovered_cases = recovered_cases.sort_values('Province/State')
confirmed_cases = confirmed_cases.sort_values('Province/State')

In [ ]:
data = {'Province/State': active_cases.index, 'Confirmed': confirmed_cases['Confirmed'].values,
       'Active': active_cases['Active'].values,'Recovered':recovered_cases['Recovered'].values,
       'Death': death_cases['Deaths'].values}
state_wise = pd.DataFrame.from_dict(data)

In [ ]:
state_wise

In [ ]:
total_active = state_wise['Active'].sum()
total_confirmed = state_wise['Confirmed'].sum()
total_recovered = state_wise['Recovered'].sum()
total_deaths = state_wise['Death'].sum()

In [ ]:
values = [total_active,total_confirmed,total_recovered,total_deaths]
labels = ['Active','Confirmed','Recovered','Deaths']

In [ ]:
explode = [0,0.1,0,0]

In [ ]:
plt.figure(figsize=(15,10))
plt.pie(values, labels=labels, shadow=True,explode=explode, startangle=270,autopct='%1.1f%%');

In [ ]:
import geopandas as gpd

In [ ]:
ls /kaggle/input/india-shape-file/

In [ ]:
df_map = gpd.read_file("/kaggle/input/india-shape-file/states_india.shp")

In [ ]:
df_map

In [ ]:
df_map["st_nm"][df_map['st_nm'] == "NCT of Delhi"] = "Delhi"
df_map = df_map.sort_values("st_nm")

In [ ]:
state_wise['Province/State'].replace({"Dadar Nagar Haveli": "Dadara & Nagar Havelli"}, inplace=True)
state_wise['Province/State'].replace({"Telangana": "Telengana"}, inplace=True)
state_wise['Province/State'].replace({"Dadra and Nagar Haveli and Daman and Diu": "Daman & Diu"}, inplace=True)

In [ ]:
state_wise

In [ ]:
df_map["st_nm"][df_map["st_nm"] == "Jammu & Kashmir"] = "Jammu and Kashmir"
df_map["st_nm"][df_map["st_nm"] == "Andaman & Nicobar Island"] = "Andaman and Nicobar Islands"
df_map["st_nm"][df_map["st_nm"] == "Telangana"] = "Telengana"
df_map["st_nm"][df_map["st_nm"] == "Arunanchal Pradesh"] = "Arunachal Pradesh"

In [ ]:
df_map

In [ ]:
state_wise_confirmed = state_wise[state_wise["Province/State"]!="Unknown"][["Province/State","Confirmed"]]

In [ ]:
state_wise_recovered = state_wise[state_wise["Province/State"]!="Unknown"][["Province/State","Recovered"]]
state_wise_active = state_wise[state_wise["Province/State"]!="Unknown"][["Province/State","Active"]]
state_wise_death = state_wise[state_wise["Province/State"]!="Unknown"][["Province/State","Death"]]


In [ ]:
state_wise_confirmed.shape

In [ ]:
merged_df = df_map.set_index('st_nm').join(state_wise_confirmed.set_index('Province/State'))
merged_df['Province/State'] = state_wise_active['Province/State'].values
merged_df = pd.merge(merged_df,state_wise_active,on='Province/State')
merged_df = pd.merge(merged_df,state_wise_death,on='Province/State')
merged_df = pd.merge(merged_df,state_wise_recovered,on='Province/State')

In [ ]:
merged_df

In [ ]:
p, ax = plt.subplots(1, figsize=(20, 15))
ax.set_title('Covid 19 confirmed cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Confirmed', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
ax.axis('off')
plt.show()

In [ ]:
p, ax = plt.subplots(1, figsize=(20, 15))
ax.set_title('Covid 19 death cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Death', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
ax.axis('off')
plt.show()

In [ ]:
p, ax = plt.subplots(1, figsize=(30, 15))
ax.set_title('Covid 19 recovered cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Recovered', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
ax.axis('off')
plt.show()

In [ ]:
p, ax = plt.subplots(1, figsize=(30, 15))
ax.set_title('Covid 19 active cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Active', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
ax.axis('off')
plt.show()

In [ ]:
p, ax = plt.subplots(2,2, figsize=(20, 15))
ax[0,0].set_title('Covid 19 confirmed cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Confirmed', cmap='Reds', linewidth=0.8, ax=ax[0,0], edgecolor='0.8', legend=True)
ax[0,0].axis('off')
ax[0,1].set_title('Covid 19 active cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Active', cmap='Reds', linewidth=0.8, ax=ax[0,1], edgecolor='0.8', legend=True)
ax[0,1].axis('off')
ax[1,0].set_title('Covid 19 recovered cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Recovered', cmap='Reds', linewidth=0.8, ax=ax[1,0], edgecolor='0.8', legend=True)
ax[1,0].axis('off')
ax[1,1].set_title('Covid 19 death cases distribution', fontdict={'fontsize': '21', 'fontweight' : '1000'})
merged_df.plot(column='Death', cmap='Reds', linewidth=0.8, ax=ax[1,1], edgecolor='0.8', legend=True)
ax[1,1].axis('off')
plt.show()

In [ ]:
output_file("covid_cases.html")
fig1 = merged_df.plot_bokeh(
    title="Confirmed cases",
    figsize=(900, 600),
    simplify_shapes=5000,
    category="Confirmed",
    show_colorbar=True,
    colormap=["blue", "yellow", "green", "red"],
    hovertool_columns=["Province/State","Confirmed"],
    tile_provider="CARTODBPOSITRON")
fig2 = merged_df.plot_bokeh(
    title="Active cases",
    figsize=(900, 600),
    simplify_shapes=5000,
    category="Active",
    show_colorbar=True,
    colormap=["blue", "yellow", "green", "red"],
    hovertool_columns=["Province/State","Active"],
    tile_provider="CARTODBPOSITRON")
fig3 = merged_df.plot_bokeh(
    title="Recovered cases",
    figsize=(900, 600),
    simplify_shapes=5000,
    category="Recovered",
    show_colorbar=True,
    colormap=["blue", "yellow", "green", "red"],
    hovertool_columns=["Province/State","Recovered"],
    tile_provider="CARTODBPOSITRON")
fig4 = merged_df.plot_bokeh(
    title="Death cases",
    figsize=(900, 600),
    simplify_shapes=5000,
    category="Death",
    show_colorbar=True,
    colormap=["blue", "yellow", "green", "red"],
    hovertool_columns=["Province/State","Death"],
    tile_provider="CARTODBPOSITRON")
p = gridplot([[fig1, fig2],[fig3, fig4]])
show(p)

In [ ]:
output_file("output_file_name.html")
save(p)